In [1]:
!pip install datasets transformers[sentencepiece] sacrebleu -q


In [2]:
import os
import sys
import transformers
import tensorflow as tf
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import AdamWeightDecay
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM

2024-05-04 11:57:12.539553: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-04 11:57:12.539648: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-04 11:57:12.655845: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
model_checkpoint = "marefa-nlp/marefa-mt-en-ar"

In [4]:
#raw_datasets = load_dataset(,split="train")
raw_datasets = load_dataset('majedk01/english-arabic-text', split='train[50%:52%]')
raw_datasets

Generating train split:   0%|          | 0/11349997 [00:00<?, ? examples/s]

Dataset({
    features: ['translation'],
    num_rows: 227000
})

In [5]:
raw_datasets

Dataset({
    features: ['translation'],
    num_rows: 227000
})

In [60]:
raw_datasets['translation'][0]

{'ar': 'وتشمل الخلفية التعليمية والمهنية لصاحب المطالبة شهادة الدكتوراة في علم الغدد الصماء والكيمياء الأحيائية وثمانية منشورات و22 ورقة بحوث.',
 'en': "The claimant's educational and professional background included a PhD degree in Biochemical Endocrinology, eight publications and 22 papers."}

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/801k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/917k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [7]:
tokenizer("Hello, this is a sentence!")

{'input_ids': [3213, 5, 64, 33, 15, 5176, 65, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [40]:
tokenizer(["Hello, this is a sentence!", "This is another sentence."])

{'input_ids': [[3213, 5, 64, 33, 15, 5176, 65, 0], [255, 33, 1144, 5176, 2, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}

In [8]:
with tokenizer.as_target_tokenizer():
    print(tokenizer(["و هذه؟"]))

{'input_ids': [[28, 71, 51, 0]], 'attention_mask': [[1, 1, 1, 1]]}


/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3935: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [7]:
max_input_length = 512
max_target_length = 512

source_lang = "en"
target_lang = "ar"


def preprocess_function(examples):
    inputs = [ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [18]:
preprocess_function(raw_datasets[:2])

{'input_ids': [[31, 10287, 16, 18, 3542, 8, 4120, 6963, 1274, 15, 55990, 6541, 11, 31688, 39857, 16958, 9027, 23345, 12368, 5, 4686, 6013, 8, 671, 6911, 2, 0], [237, 106, 105, 15, 28406, 11, 14404, 41702, 4078, 5417, 701, 12219, 5, 203, 632, 3, 9927, 8, 3, 3313, 6, 198, 1621, 5, 155, 53, 6209, 9, 1016, 1721, 296, 2673, 8, 9275, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[3736, 15501, 5814, 36886, 17017, 5411, 9358, 9578, 37, 35, 10, 3471, 13474, 141, 31903, 746, 150, 13722, 9181, 20640, 28, 31397, 15084, 28, 4938, 4894, 12228, 2, 0], [1240, 19120, 37, 26, 10, 48622, 20640, 576, 149, 12219, 7, 5568, 241, 21, 1550, 32398, 28, 13885, 12228, 38, 7, 101, 16670, 77, 89, 13676, 10178, 17, 5365, 77, 200, 10, 551, 57, 107, 2673, 28, 28179, 2, 0]]}

In [8]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/227000 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3935: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [9]:
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint,from_pt=True)

pytorch_model.bin:   0%|          | 0.00/306M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFMarianMTModel: ['lm_head.weight']
- This IS expected if you are initializing TFMarianMTModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFMarianMTModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2

In [10]:
batch_size = 16
learning_rate = 2e-5
weight_decay = 0.01
num_train_epochs = 1

In [12]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

In [12]:
tokenized_datasets

Dataset({
    features: ['translation', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 227000
})

In [13]:
train_dataset = model.prepare_tf_dataset(
    tokenized_datasets,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=data_collator,
)

In [14]:
optimizer = AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)
model.compile(optimizer=optimizer)

In [15]:
model.fit(train_dataset, validation_data=train_dataset, epochs=1)


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
14187/14187 [==============================] - ETA: 0s - loss: 1.5528

KeyboardInterrupt: 

In [19]:
model.save_pretrained("tf_model/")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[62801]], 'forced_eos_token_id': 0}


In [20]:
import zipfile
file_path = "/kaggle/working/Model.h5"

# Path where you want to save the zip file
zip_file_path = "/kaggle/working/Model.zip"
with zipfile.ZipFile(zip_file_path, 'w') as zipf:
    # Add the file to the zip
    zipf.write(file_path, arcname='Model')

print("Zip file created successfully!")

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/working/Model.h5'

In [49]:
from IPython.display import FileLink
FileLink(r'Model.zip')

/kaggle/working/Model.zip

In [21]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = TFAutoModelForSeq2SeqLM.from_pretrained("tf_model/")

/opt/conda/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at tf_model/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


In [26]:
input_text  = """Russia, with its vast expanses and rich cultural heritage, has a history as diverse and complex as its landscape. From the earliest Slavic settlements to the mighty Tsarist Empire and the Soviet era, the story of Russia is one of triumphs, struggles, and constant evolution.

The roots of Russian civilization trace back to the 9th century when Slavic tribes began to settle along the Eastern European plains. These early communities laid the groundwork for what would become the Russian state, adopting Christianity from Byzantium and establishing the first centralized political entities.

The medieval period saw the rise of the powerful Kievan Rus, a federation of East Slavic tribes under the rule of the Varangians. Kiev, with its grand cathedrals and bustling trade routes, emerged as a center of culture and commerce in Eastern Europe. However, internal strife and external invasions eventually weakened Kievan Rus, leading to its fragmentation and the emergence of independent principalities.

In the 13th century, the Mongol invasion swept across Eurasia, bringing much of Russia under the control of the Golden Horde. Despite the Mongol yoke, Russian principalities continued to develop, with Moscow emerging as a prominent center of resistance and eventually asserting its dominance over neighboring territories.

The rise of Moscow laid the foundation for the Tsardom of Russia, which emerged in the 16th century under Ivan the Terrible. Ivan's reign saw the expansion of Russian territory through conquest and diplomacy, as well as the consolidation of power under the autocratic rule of the Tsar. The Romanov dynasty, which came to power in 1613, further solidified Russian control over its vast domain and presided over a period of cultural flourishing known as the Russian Renaissance.

The 18th century witnessed Russia's transformation into a European power under the reign of Peter the Great. Inspired by Western models, Peter implemented sweeping reforms aimed at modernizing the Russian state and expanding its influence abroad. His successors continued this trend, with Catherine the Great presiding over a golden age of Russian literature, art, and architecture."""

tokenized = tokenizer([input_text], return_tensors='np')
out = model.generate(**tokenized, max_length=128)
print(out)

tf.Tensor(
[[62801    28 28824 27283 40833 13469     7    75 23536 15121  3772    49
     28  1112  4575 30371    49  7393     7    27  3078  8770    87  2987
    565 31963 13280    37   965    35   121 19097    17  6765  2220 21986
   2691  2969     7   224 38115    71  3099   804 15717  4150  1209 20222
    375 13469     7  1314  3357 22453    12  1368 15401 10391 22589 11576
   6617   980  6187     7 33823   978 47312  2990 41719   793  2247    47
  12227  1879  2529    18 17355     7   654  7060    12 31963 13280    37
    965    35  2969  1028  3763  7602  8019  4027    80     7  2247    47
   3392     7    75 29577  1391    49  6496    74  3732  5452 30590    49
   1664   193 41969    35     7    41 15816 46461   202 15353  8983    10
   1918  2969     7  1030    58 26946    89     0]], shape=(1, 128), dtype=int32)


In [27]:
with tokenizer.as_target_tokenizer():
    print(tokenizer.decode(out[0], skip_special_tokens=True))

وتعود جذور الحضارة الروسية، مع اتساع رقعتها وثراء تراثها الثقافي، إلى القرن التاسع الذي بدأت فيه القبائل السلافية تستقر على طول السهول الأوروبية الشرقية، وقد أرست هذه المجتمعات المحلية المبكرة الأساس لما سيصبح الدولة الروسية، إذ اعتمدت المسيحية من بيزنطة وأنشأت أولى الكيانات السياسية المركزية، وشهدت فترة القرون الوسطى صعود مجموعة كييفان Rus القوية، وهي اتحاد من القبائل السلافية الشرقية تحت حكم الفارانجيين، كيييف، مع كبرياتها الكاتدرائية وطرقها التجارية المتعثرة، التي ظهرت كمركز للثقافة والتجارة في أوروبا الشرقية، ومع ذلك أضعفت
